# Prepare dataset filtering

In [1]:
import os
import shutil
import pandas as pd
from collections import defaultdict
import random

In [2]:
base_dataset_path = 'dataset' # Asume que tu notebook está al mismo nivel que la carpeta 'dataset'

train_path_original = os.path.join(base_dataset_path, 'train')
test_path_original = os.path.join(base_dataset_path, 'test')
valid_path_original = os.path.join(base_dataset_path, 'valid')

print(f"Ruta de entrenamiento original: {train_path_original}")
print(f"Ruta de prueba original: {test_path_original}")
print(f"Ruta de validación original: {valid_path_original}")

Ruta de entrenamiento original: dataset/train
Ruta de prueba original: dataset/test
Ruta de validación original: dataset/valid


In [3]:
species_counts = defaultdict(int)

# Recorrer cada carpeta de especie dentro de 'train'
for species_folder in os.listdir(train_path_original):
    species_folder_path = os.path.join(train_path_original, species_folder)
    if os.path.isdir(species_folder_path): # Asegurarse de que sea una carpeta
        # Contar el número de archivos (imágenes) en la carpeta de la especie
        num_images = len([name for name in os.listdir(species_folder_path) if os.path.isfile(os.path.join(species_folder_path, name))])
        species_counts[species_folder] = num_images

# Convertir a un DataFrame para fácil visualización y ordenamiento
df_species_counts = pd.DataFrame(species_counts.items(), columns=['Species', 'Count'])
df_species_counts = df_species_counts.sort_values(by='Count', ascending=False).reset_index(drop=True)

print("Cantidad de muestras por especie en el conjunto de entrenamiento (ordenado de mayor a menor):")
print(df_species_counts.head()) # Mostrar las primeras para verificar
print(f"\nTotal de especies encontradas: {len(df_species_counts)}")

Cantidad de muestras por especie en el conjunto de entrenamiento (ordenado de mayor a menor):
            Species  Count
0    MOURNING CLOAK    187
1  GREEN HAIRSTREAK    176
2       BROWN ARGUS    169
3  BROOKES BIRDWING    165
4     SLEEPY ORANGE    153

Total de especies encontradas: 100


In [4]:
top_30_species = df_species_counts.head(30)['Species'].tolist()

print("\nLas 30 especies con la mayor cantidad de muestras son:")
for i, species in enumerate(top_30_species):
    print(f"{i+1}. {species}")

print(f"\nTotal de especies seleccionadas: {len(top_30_species)}")


Las 30 especies con la mayor cantidad de muestras son:
1. MOURNING CLOAK
2. GREEN HAIRSTREAK
3. BROWN ARGUS
4. BROOKES BIRDWING
5. SLEEPY ORANGE
6. CHALK HILL BLUE
7. WHITE LINED SPHINX MOTH
8. HUMMING BIRD HAWK MOTH
9. ATALA
10. BROWN SIPROETA
11. ARCIGERA FLOWER MOTH
12. SCARCE SWALLOW
13. BANDED ORANGE HELICONIAN
14. CRECENT
15. MILBERTS TORTOISESHELL
16. RED CRACKER
17. ROSY MAPLE MOTH
18. ORANGE TIP
19. HERCULES MOTH
20. BANDED TIGER MOTH
21. Iphiclus sister
22. EASTERN PINE ELFIN
23. EMPEROR GUM MOTH
24. CHECQUERED SKIPPER
25. GREAT JAY
26. DANAID EGGFLY
27. COPPER TAIL
28. EASTERN COMA
29. BIRD CHERRY ERMINE MOTH
30. CLEOPATRA

Total de especies seleccionadas: 30


# Create filtered dataset

In [7]:
filtered_dataset_path = 'filtered_dataset'
filtered_train_path = os.path.join(filtered_dataset_path, 'train')
filtered_test_path = os.path.join(filtered_dataset_path, 'test')
filtered_valid_path = os.path.join(filtered_dataset_path, 'valid')

# Crear las carpetas base del dataset filtrado si no existen
os.makedirs(filtered_train_path, exist_ok=True)
os.makedirs(filtered_test_path, exist_ok=True)
os.makedirs(filtered_valid_path, exist_ok=True)

print(f"\nCreando la estructura de carpetas en: {filtered_dataset_path}")

# Función para copiar carpetas de especies
def copy_species_folders(source_base, destination_base, species_list):
    for species in species_list:
        source_species_path = os.path.join(source_base, species)
        destination_species_path = os.path.join(destination_base, species)

        if os.path.exists(source_species_path):
            shutil.copytree(source_species_path, destination_species_path, dirs_exist_ok=True)
            print(f"Copiado: {source_species_path} a {destination_species_path}")
        else:
            print(f"Advertencia: La carpeta de origen '{source_species_path}' no existe. Creando destino vacío.")
            os.makedirs(destination_species_path, exist_ok=True)

# Copiar las 30 especies seleccionadas a la nueva estructura
print("Copiando especies al nuevo directorio 'train' filtrado...")
copy_species_folders(train_path_original, filtered_train_path, top_30_species)

print("Copiando especies al nuevo directorio 'test' filtrado...")
copy_species_folders(test_path_original, filtered_test_path, top_30_species)

print("Copiando especies al nuevo directorio 'valid' filtrado...")
copy_species_folders(valid_path_original, filtered_valid_path, top_30_species)

print("\nDuplicación del dataset filtrado completada.")


Creando la estructura de carpetas en: filtered_dataset
Copiando especies al nuevo directorio 'train' filtrado...
Copiado: dataset/train/MOURNING CLOAK a filtered_dataset/train/MOURNING CLOAK
Copiado: dataset/train/GREEN HAIRSTREAK a filtered_dataset/train/GREEN HAIRSTREAK
Copiado: dataset/train/BROWN ARGUS a filtered_dataset/train/BROWN ARGUS
Copiado: dataset/train/BROOKES BIRDWING a filtered_dataset/train/BROOKES BIRDWING
Copiado: dataset/train/SLEEPY ORANGE a filtered_dataset/train/SLEEPY ORANGE
Copiado: dataset/train/CHALK HILL BLUE a filtered_dataset/train/CHALK HILL BLUE
Copiado: dataset/train/WHITE LINED SPHINX MOTH a filtered_dataset/train/WHITE LINED SPHINX MOTH
Copiado: dataset/train/HUMMING BIRD HAWK MOTH a filtered_dataset/train/HUMMING BIRD HAWK MOTH
Copiado: dataset/train/ATALA a filtered_dataset/train/ATALA
Copiado: dataset/train/BROWN SIPROETA a filtered_dataset/train/BROWN SIPROETA
Copiado: dataset/train/ARCIGERA FLOWER MOTH a filtered_dataset/train/ARCIGERA FLOWER MOT

# Move 20 samples to `test` folder

In [8]:
num_samples_to_move = 20

print(f"\nMoviendo {num_samples_to_move} muestras de 'train' a 'test' para cada una de las 30 especies...")

for species in top_30_species:
    source_species_train_path = os.path.join(filtered_train_path, species)
    destination_species_test_path = os.path.join(filtered_test_path, species)

    # Asegurarse de que las carpetas de destino en test existan
    os.makedirs(destination_species_test_path, exist_ok=True)

    # Obtener todas las imágenes en la carpeta de la especie en train
    images_in_train = [f for f in os.listdir(source_species_train_path) if os.path.isfile(os.path.join(source_species_train_path, f))]

    # Seleccionar aleatoriamente las imágenes a mover
    if len(images_in_train) >= num_samples_to_move:
        samples_to_move = random.sample(images_in_train, num_samples_to_move)
    else:
        samples_to_move = images_in_train
        print(f"Advertencia: La especie '{species}' solo tiene {len(images_in_train)} muestras en train. Moviendo todas.")
        if len(images_in_train) == 0:
            print(f"  No hay muestras para mover para '{species}'.")
            continue

    # Mover las imágenes seleccionadas
    for img_name in samples_to_move:
        source_img_path = os.path.join(source_species_train_path, img_name)
        destination_img_path = os.path.join(destination_species_test_path, img_name)
        shutil.move(source_img_path, destination_img_path)
        print(f"  Movido: {img_name} de {species} a test.")

print(f"\nMovimiento de {num_samples_to_move} muestras por especie completado.")


Moviendo 20 muestras de 'train' a 'test' para cada una de las 30 especies...
  Movido: 134.jpg de MOURNING CLOAK a test.
  Movido: 169.jpg de MOURNING CLOAK a test.
  Movido: 101.jpg de MOURNING CLOAK a test.
  Movido: 074.jpg de MOURNING CLOAK a test.
  Movido: 086.jpg de MOURNING CLOAK a test.
  Movido: 030.jpg de MOURNING CLOAK a test.
  Movido: 111.jpg de MOURNING CLOAK a test.
  Movido: 040.jpg de MOURNING CLOAK a test.
  Movido: 072.jpg de MOURNING CLOAK a test.
  Movido: 023.jpg de MOURNING CLOAK a test.
  Movido: 140.jpg de MOURNING CLOAK a test.
  Movido: 172.jpg de MOURNING CLOAK a test.
  Movido: 136.jpg de MOURNING CLOAK a test.
  Movido: 048.jpg de MOURNING CLOAK a test.
  Movido: 061.jpg de MOURNING CLOAK a test.
  Movido: 064.jpg de MOURNING CLOAK a test.
  Movido: 129.jpg de MOURNING CLOAK a test.
  Movido: 036.jpg de MOURNING CLOAK a test.
  Movido: 162.jpg de MOURNING CLOAK a test.
  Movido: 173.jpg de MOURNING CLOAK a test.
  Movido: 034.jpg de GREEN HAIRSTREAK a te

In [9]:
print("\nVerificando las nuevas cantidades de muestras en 'filtered_dataset':")

def count_images_in_filtered_dataset(base_path, species_list):
    counts = defaultdict(lambda: {'train': 0, 'test': 0, 'valid': 0})
    for species in species_list:
        train_path = os.path.join(base_path, 'train', species)
        test_path = os.path.join(base_path, 'test', species)
        valid_path = os.path.join(base_path, 'valid', species)

        if os.path.exists(train_path):
            counts[species]['train'] = len([f for f in os.listdir(train_path) if os.path.isfile(os.path.join(train_path, f))])
        if os.path.exists(test_path):
            counts[species]['test'] = len([f for f in os.listdir(test_path) if os.path.isfile(os.path.join(test_path, f))])
        if os.path.exists(valid_path):
            counts[species]['valid'] = len([f for f in os.listdir(valid_path) if os.path.isfile(os.path.join(valid_path, f))])
    return counts

final_counts = count_images_in_filtered_dataset(filtered_dataset_path, top_30_species)

df_final_counts = pd.DataFrame.from_dict(final_counts, orient='index')
df_final_counts.index.name = 'Species'
df_final_counts = df_final_counts.reset_index()

print(df_final_counts)

total_train = df_final_counts['train'].sum()
total_test = df_final_counts['test'].sum()
total_valid = df_final_counts['valid'].sum()

print(f"\nTotal de imágenes en 'filtered_train': {total_train}")
print(f"Total de imágenes en 'filtered_test': {total_test}")
print(f"Total de imágenes en 'filtered_valid': {total_valid}")
print(f"Total general de imágenes en el dataset filtrado: {total_train + total_test + total_valid}")


Verificando las nuevas cantidades de muestras en 'filtered_dataset':
                     Species  train  test  valid
0             MOURNING CLOAK    167    25      5
1           GREEN HAIRSTREAK    156    25      5
2                BROWN ARGUS    149    25      5
3           BROOKES BIRDWING    145    25      5
4              SLEEPY ORANGE    133    25      5
5            CHALK HILL BLUE    129    25      5
6    WHITE LINED SPHINX MOTH    123    25      5
7     HUMMING BIRD HAWK MOTH    123    25      5
8                      ATALA    123    25      5
9             BROWN SIPROETA    121    25      5
10      ARCIGERA FLOWER MOTH    120    25      5
11            SCARCE SWALLOW    119    25      5
12  BANDED ORANGE HELICONIAN    119    25      5
13                   CRECENT    118    25      5
14    MILBERTS TORTOISESHELL    117    25      5
15               RED CRACKER    117    25      5
16           ROSY MAPLE MOTH    117    25      5
17                ORANGE TIP    117    25      5

In [10]:
print("\nGenerando el archivo CSV con los metadatos del dataset filtrado...")

# Definir la ruta donde se guardará el CSV
csv_output_path = os.path.join(filtered_dataset_path, 'filtered_dataset_metadata.csv')

# Crear un diccionario para mapear el nombre de la especie a un índice numérico
species_to_index = {species: i for i, species in enumerate(top_30_species)}

data = []

# Recorrer los conjuntos (train, test, valid)
for dataset_type in ['train', 'test', 'valid']:
    current_dataset_path = os.path.join(filtered_dataset_path, dataset_type)

    # Recorrer cada especie dentro del conjunto actual
    for species_name in os.listdir(current_dataset_path):
        species_folder_path = os.path.join(current_dataset_path, species_name)

        if os.path.isdir(species_folder_path) and species_name in species_to_index:
            class_index = species_to_index[species_name]

            # Recorrer cada imagen en la carpeta de la especie
            for image_name in os.listdir(species_folder_path):
                image_path = os.path.join(species_folder_path, image_name)

                if os.path.isfile(image_path):
                    # 'filtered_dataset_path' es la base de referencia para las rutas relativas.
                    relative_filepath = os.path.relpath(image_path, start=filtered_dataset_path)

                    data.append({
                        'class_index': class_index,
                        'filepaths': relative_filepath,
                        'labels': species_name,
                        'data_set': dataset_type
                    })

# Crear un DataFrame de pandas
df_metadata = pd.DataFrame(data)

# Guardar el DataFrame en un archivo CSV
df_metadata.to_csv(csv_output_path, index=False)

print(f"\nCSV de metadatos generado exitosamente en: {csv_output_path}")
print("\nLas primeras 5 filas del CSV (con rutas relativas):")
print(df_metadata.head())
print(f"\nTotal de filas en el CSV: {len(df_metadata)}")

# Opcional: Verificar la distribución de los datasets en el CSV
print("\nDistribución de imágenes por conjunto en el CSV:")
print(df_metadata['data_set'].value_counts())


Generando el archivo CSV con los metadatos del dataset filtrado...

CSV de metadatos generado exitosamente en: filtered_dataset/filtered_dataset_metadata.csv

Las primeras 5 filas del CSV (con rutas relativas):
   class_index                  filepaths       labels data_set
0           15  train/RED CRACKER/016.jpg  RED CRACKER    train
1           15  train/RED CRACKER/002.jpg  RED CRACKER    train
2           15  train/RED CRACKER/003.jpg  RED CRACKER    train
3           15  train/RED CRACKER/017.jpg  RED CRACKER    train
4           15  train/RED CRACKER/029.jpg  RED CRACKER    train

Total de filas en el CSV: 4593

Distribución de imágenes por conjunto en el CSV:
data_set
train    3693
test      750
valid     150
Name: count, dtype: int64
